[View in Colaboratory](https://colab.research.google.com/github/mogamin/chainer-examples/blob/master/chainer_PotatoChips_Classification.ipynb)

In [0]:
# まずはデータのダウンロード、展開
#!curl -L "https://rebrand.ly/dllab2018-hackathon-cv" -o data.tar.gz
#!gzip -d -c data.tar.gz | tar xf -

In [0]:
# デフォルトでcolabに入っていない使用するパッケージをインストール
#!pip install chainer
#!pip install chainercv

In [0]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer.datasets import LabeledImageDataset
from chainer.datasets import TransformDataset

In [0]:
from chainercv.transforms import resize

# 画像データの前処理変換
class ImageTransform(object):
    def __call__(self, in_data):
        img, label = in_data
        #_, H, W = img.shape
        #img = random_sized_crop(img)
        img = resize(img, (224, 224))
        #img = random_flip(img, x_random=True)
        return img, label

In [0]:
# データのロード
train = LabeledImageDataset('data/train/train_labels.txt', root='data/train/images')
train = TransformDataset(train, ImageTransform())
valid = LabeledImageDataset('data/valid/valid_labels.txt', root='data/valid/images')
valid = TransformDataset(valid, ImageTransform())

In [0]:
class PotatoMLP(chainer.Chain):
  def __init__(self, n_mid_units=5, n_class=9):
    super().__init__()
    with self.init_scope():
        self.fc1 = L.Linear(None, n_mid_units)
        self.fc2 = L.Linear(None, n_mid_units)
        self.fc3 = L.Linear(None, n_class)
  
  def __call__(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.fc2(h)
    h = F.relu(h)
    return h   
  

In [0]:
# イテレータの準備
BATCHSIZE = 10
train_iter = chainer.iterators.SerialIterator(train, BATCHSIZE)
valid_iter = chainer.iterators.SerialIterator(valid, BATCHSIZE, repeat=False, shuffle=False)

In [8]:
#
mlp = PotatoMLP(224, 9)
model = L.Classifier(mlp)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [9]:
#
from chainer.training import extensions

EPOCH = 10
updater = chainer.training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = chainer.training.Trainer(updater, (EPOCH, 'epoch'), out='result/potato')
trainer.extend(extensions.Evaluator(valid_iter, model, device=-1))
trainer.extend(extensions.LogReport(trigger=(1,'epoch')))
trainer.extend(extensions.PrintReport(['epoch','main/accuracy','validation/main/accuracy','main/loss','validation/main/loss','elapsed_time']), trigger=(1,'epoch'))
trainer.extend(extensions.ProgressBar(update_interval=10))
trainer.run()

     total [..................................................]  0.09%
this epoch [..................................................]  0.95%
        10 iter, 0 epoch / 10 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  0.19%
this epoch [..................................................]  1.90%
        20 iter, 0 epoch / 10 epochs
    1.1626 iters/sec. Estimated time to finish: 2:30:44.972888.
     total [..................................................]  0.28%
this epoch [#.................................................]  2.85%
        30 iter, 0 epoch / 10 epochs
    1.1883 iters/sec. Estimated time to finish: 2:27:20.982939.
     total [..................................................]  0.38%
this epoch [#.................................................]  3.80%
        40 iter, 0 epoch / 10 epochs
    1.1958 iters/sec. Estimated time to finish: 2:26:17.448781.
     total [.......................

KeyboardInterrupt: ignored